Today's challenge didn't seem to lend itself too well to pandas. oh well.
essentially, iterate over each line.

part 1: find all the numbers, then check up, down, left, right of them for symbols

part 2: find a star, then check up, down, left, right of them for numbers

I'd love to but I don't have the will to refactor any of this

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('elf-input.txt',header=None)

In [3]:
symbols = '|\\'.join(df[0].str.extractall('([^.0-9a-z])')[0].drop_duplicates().to_list())
symbols = '\\' + symbols

In [4]:
# Create an empty list 
res_list =[]

# Iterate over each line 
for index, line in df.iterrows(): 
    # find the numbers in the row
    nb_list = line.str.extractall('(\d+)')[0].to_list()
    my_pos_in_str = 0

    for nb in nb_list:
        valid_nb = False
        #find the index of the number
        nb_pos_in_str = line.str.find(nb, start=my_pos_in_str)[0]
        my_pos_in_str = nb_pos_in_str + len(nb)
        slice_start = nb_pos_in_str-1
        if slice_start < 0:
            slice_start = 0
        slice_end = my_pos_in_str+1
        #check row above for symbols
        if (index-1) in df.index:
            valid_nb = valid_nb or df.loc[index-1].str.slice(start=slice_start, stop=slice_end).str.contains(symbols)[0]
        #check row below for symbols
        if (index+1) in df.index:
            valid_nb = valid_nb or df.loc[index+1].str.slice(start=slice_start, stop=slice_end).str.contains(symbols)[0]
        #check char to the left for symbols
        if nb_pos_in_str > 0:
            valid_nb = valid_nb or line.str[slice_start].str.contains(symbols)[0]
        #check char to the right for symbols
        if not line.str[my_pos_in_str].isna().any():
            valid_nb = valid_nb or line.str[my_pos_in_str].str.contains(symbols)[0]
        #if symbols are detected in any of the above
            #keep number
        if valid_nb:
            res_list.append(nb)

res_df = pd.DataFrame(res_list)

In [5]:
res_df.astype(int).sum()

0    539713
dtype: int64

Part 2

In [11]:
# Create an empty list 
res_list =[]
max_nb_len = pd.DataFrame(nb_list).apply(lambda x: len(x[0]))[0].max()

# Iterate over each line 
for index, line in df.iterrows(): 
    # find the number of stars in the row
    nb_stars_in_line = line.str.count('(\*)')[0]
    my_pos_in_str = 0
    
    for star in range(nb_stars_in_line):
        tmp_list=[]
        #find the index of the next star
        nb_pos_in_str = line.str.find('*', start=my_pos_in_str)[0]
        my_pos_in_str = nb_pos_in_str + 1
        #check numbers above
        if (index-1) in df.index:
            my_pos_in_str_above = 0
            for nb in df.loc[index-1].str.extractall('(\d+)')[0].dropna().to_list():
                nb_pos_in_str_above = df.loc[index-1].str.find(nb, start=my_pos_in_str_above)[0]
                my_pos_in_str_above = nb_pos_in_str_above + len(nb)
                if nb_pos_in_str in range(nb_pos_in_str_above-1, my_pos_in_str_above+1):
                    tmp_list.append(nb)
        #check numbers below
        if (index+1) in df.index:
            my_pos_in_str_below = 0
            for nb in df.loc[index+1].str.extractall('(\d+)')[0].dropna().to_list():
                nb_pos_in_str_below = df.loc[index+1].str.find(nb, start=my_pos_in_str_below)[0]
                my_pos_in_str_below = nb_pos_in_str_below + len(nb)
                if nb_pos_in_str in range(nb_pos_in_str_below-1, my_pos_in_str_below+1):
                    tmp_list.append(nb)
        #look to the left for a number
        slice_start = nb_pos_in_str - max_nb_len -1
        if nb_pos_in_str < 0:
            slice_start = 0
        tmp_res_list = line.str.slice(start=slice_start, stop=nb_pos_in_str).str.extractall('(\d+)')[0].dropna().to_list()
        if len(tmp_res_list) > 0:
            nb = tmp_res_list[len(tmp_res_list)-1]
            nb_pos_in_left_str = line.str.find(nb, start=slice_start)[0]
            if ((nb_pos_in_left_str > -1) and (nb_pos_in_left_str+len(nb) == nb_pos_in_str)):
                tmp_list.append(nb)
        #look to the right for a number
        slice_start = nb_pos_in_str 
        slice_end = slice_start + max_nb_len+2
        if slice_end > line.str.len()[0]:
            slice_end = line.str.len()[0]
        tmp_res_list = line.str.slice(start=slice_start, stop=slice_end).str.extractall('(\d+)')[0].dropna().to_list()
        if len(tmp_res_list) > 0 :
            nb = tmp_res_list[0]
            nb_pos_in_right_str = line.str.find(nb, start=nb_pos_in_str)[0]
            if ((nb_pos_in_right_str > -1) and (nb_pos_in_right_str == nb_pos_in_str+1)):
                tmp_list.append(nb)
        #if only 2 numbers
            #multiply
        if len(tmp_list) == 2:
            res_list.append(int(tmp_list[0])*int(tmp_list[1]))
#sum gear ratios


In [12]:
res_df = pd.DataFrame(res_list)
res_df.sum()

0    84159075
dtype: int64